In [17]:
# 실시간 날씨 데이터 조회

import requests
import json

# OpenWeatherMap API 키
API_KEY = "13a24ecc3a2785b9652b8816b02a8e55"

def get_lat_lon_from_ip():
    """IP 기반으로 위도와 경도, 도시명 반환"""
    try:
        response = requests.get("https://ipinfo.io/json", timeout=5)
        data = response.json()
        lat, lon = map(float, data["loc"].split(","))
        city = data.get("city", "알 수 없음")
        print(f"📍 자동 위치 감지: {city} (위도 {lat}, 경도 {lon})")
        return lat, lon, city
    except Exception as e:
        print("❌ IP 기반 위치 조회 실패:", e)
        return None, None, None

def get_lat_lon(city_name):
    """사용자 입력 도시명을 위도/경도로 변환"""
    geo_url = f"https://api.openweathermap.org/geo/1.0/direct?q={city_name}&limit=1&appid={API_KEY}"
    response = requests.get(geo_url)
    
    if response.status_code == 200 and response.json():
        data = response.json()[0]
        return data["lat"], data["lon"]
    else:
        print("❌ 입력한 도시의 위치를 찾을 수 없습니다.")
        return None, None
        
def get_weather(lat, lon):
    """위도와 경도로 날씨 데이터 가져오기"""
    weather_url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_KEY}&units=metric"
    response = requests.get(weather_url)

    if response.status_code == 200:
        return response.json()
    else:
        print("❌ 날씨 데이터를 가져오는 데 실패했습니다.")
        return None

# ✅ 위치 조회 시도: 자동 → 실패 시 수동
lat, lon, city = get_lat_lon_from_ip()

# 자동 조회 실패 시 사용자에게 도시명을 입력받기
if lat is None or lon is None:
    city = input("🌍 자동 위치 조회 실패! 직접 도시명을 입력하세요: ")
    lat, lon = get_lat_lon(city)

# 날씨 데이터 조회
if lat and lon:
    weather_data = get_weather(lat, lon)

    if weather_data:
        # 🌤️ 텍스트 표현 (UI용)
        weather = weather_data["weather"][0]["main"]
        description = weather_data["weather"][0]["description"]

        # 🌡️ 감성 벡터에 필요한 정량 데이터 추출
        temperature = weather_data["main"]["temp"]                      # 현재 기온
        feels_like = weather_data["main"]["feels_like"]                # 체감 기온
        humidity = weather_data["main"]["humidity"]                    # 습도 (%)
        cloudiness = weather_data["clouds"]["all"]                     # 구름 양 (%)
        wind_speed = weather_data["wind"]["speed"]                     # 풍속 (m/s)
        visibility = weather_data.get("visibility", 10000)             # 가시거리 (m)

        # 🕒 시간 기반 데이터
        sunrise = weather_data["sys"]["sunrise"]                       # 일출 시간 (UNIX timestamp)
        sunset = weather_data["sys"]["sunset"]                         # 일몰 시간 (UNIX timestamp)
        current_time = weather_data["dt"]                              # 현재 시각 (UNIX timestamp)

        # 🖨️ 출력 확인
        print(f"\n📍 [ {city} ] 의 날씨 정보")
        print(f"🌤️ 상태: {weather} ({description})")
        print(f"🌡️ 기온: {temperature}°C")
        print(f"🥶 체감 기온: {feels_like}°C")
        print(f"💧 습도: {humidity}%")
        print(f"☁️ 구름 양: {cloudiness}%")
        print(f"🌬️ 풍속: {wind_speed} m/s")
        print(f"👁️ 가시거리: {visibility} m")
        print(f"🌅 일출: {sunrise} | 🌇 일몰: {sunset} | 🕒 현재 시간: {current_time}")
    else:
        print("❌ 실시간 날씨 데이터를 가져오는 데 실패했습니다.")
else:
    print("❌ 위치 정보를 확인할 수 없어 실시간 날씨 조회를 진행할 수 없습니다.")

📍 자동 위치 감지: Cheonan (위도 36.8065, 경도 127.1522)

📍 [ Cheonan ] 의 날씨 정보
🌤️ 상태: Clouds (overcast clouds)
🌡️ 기온: 19.79°C
🥶 체감 기온: 20.36°C
💧 습도: 97%
☁️ 구름 양: 92%
🌬️ 풍속: 0.33 m/s
👁️ 가시거리: 10000 m
🌅 일출: 1747772385 | 🌇 일몰: 1747823788 | 🕒 현재 시간: 1747761829


In [18]:
# 📦 저장을 위한 모듈
import json
import os
import math
from datetime import datetime

# 🕒 날짜 문자열 생성
date_str = datetime.now().strftime("%Y%m%d")

# 🔁 NaN → None 변환 함수 정의
def sanitize_nan(data):
    return {
        k: (None if isinstance(v, float) and math.isnan(v) else v)
        for k, v in data.items()
    }

# ✅ 저장 파일명
filename = f"realtime_weather_{city}_{date_str}.json"

# ❗ NaN 정리한 후 저장 (weather_data 기준!)
weather_data_cleaned = sanitize_nan(weather_data)

# 📥 JSON 파일 저장
try:
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(weather_data_cleaned, f, ensure_ascii=False, indent=4)

    if os.path.exists(filename):
        print(f"✅ 파일이 성공적으로 저장되었습니다: {filename}")
    else:
        print("⚠️ 저장에 실패했습니다. 파일이 존재하지 않습니다.")

except Exception as e:
    print(f"❌ 오류 발생: {e}")



✅ 파일이 성공적으로 저장되었습니다: realtime_weather_Cheonan_20250521.json


In [19]:
print(json.dumps(weather_data, indent=4))

{
    "coord": {
        "lon": 127.1522,
        "lat": 36.8065
    },
    "weather": [
        {
            "id": 804,
            "main": "Clouds",
            "description": "overcast clouds",
            "icon": "04n"
        }
    ],
    "base": "stations",
    "main": {
        "temp": 19.79,
        "feels_like": 20.36,
        "temp_min": 19.79,
        "temp_max": 19.79,
        "pressure": 1010,
        "humidity": 97,
        "sea_level": 1010,
        "grnd_level": 997
    },
    "visibility": 10000,
    "wind": {
        "speed": 0.33,
        "deg": 131,
        "gust": 0.32
    },
    "clouds": {
        "all": 92
    },
    "dt": 1747761829,
    "sys": {
        "country": "KR",
        "sunrise": 1747772385,
        "sunset": 1747823788
    },
    "timezone": 32400,
    "id": 1845759,
    "name": "Cheonan",
    "cod": 200
}
